# Filter a recording

## Introduction

This notebook demonstrates how to efficiently read a recording, process every frame, and write out an altered version without undesired data.

## Setup

Provide input and output file paths:

In [1]:
original_file = "nanotube-example-recording.nanover.zip"
filtered_file = "nanotube-example-recording--FILTERED.nanover.zip"

Define the main function that does all the reading a writing:

In [2]:
from nanover.recording.reading import message_events_from_recording
from nanover.recording.writing import record_messages

def filter_recording(*, input_file, output_file, filter_func):
    # take events from recording
    original_events = message_events_from_recording(input_file)
    # filter each event
    filtered_events = (filter_func(event) for event in original_events)
    # record filtered events except empty messages
    record_messages(output_file, (event for event in filtered_events if event.message))

Define the filter function that takes a message event from the recording and strips out unwanted data:

In [3]:
from nanover.trajectory import keys

def strip_velocities_forces(event):
    # get frame update or dummy dict
    frame_dict = event.message.get("frame", {})
    # remove keys from frame update
    frame_dict.pop(keys.PARTICLE_VELOCITIES, None)
    frame_dict.pop(keys.PARTICLE_FORCES_SYSTEM, None)
    return event

## Execution

Run the main function with the input path, output path, and filter function to use:

In [4]:
filter_recording(input_file=original_file, output_file=filtered_file, filter_func=strip_velocities_forces)